In [1]:
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from common.multi_layer_net import MultiLayerNet
from collections import OrderedDict
from common.optimizer import *
from common.functions import sigmoid, tanh, relu
from common.multi_layer_net_extend import MultiLayerNetExtend
from common.util import shuffle_dataset, smooth_curve